In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)



Mounted at /content/drive


In [ ]:
!python -m spacy download fr_core_news_sm

     |████████████████████████████████| 14.7 MB 11.1 MB/s 
  Created wheel for fr-core-news-sm: filename=fr_core_news_sm-2.2.5-py3-none-any.whl size=14727026 sha256=8cf2bec68413cf999c4bf8bc16c2230f4651ddef9c14518e34cd5b0086338bf9
  Stored in directory: /tmp/pip-ephem-wheel-cache-pl4t_mlh/wheels/c9/a6/ea/0778337c34660027ee67ef3a91fb9d3600b76777a912ea1c24
Successfully built fr-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')


In [ ]:
chemincorpus = "/content/drive/MyDrive/Stage Defense/Corpus/"
cheminw2v = chemincorpus + 'W2V_EN_mentions_Defense.csv'
cheminbow = chemincorpus + 'BoW_EN_mentions_Defense.csv'
cheminfeatures = chemincorpus +"features_EN_mentions_Defense.csv"


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:

import spacy
nlp = spacy.load('en')

Fonctions utiles

In [ ]:
def equilibre_classes(X,y):
  '''Retire aléatoirement des exemples des classes surnnuméraires, à la fois dans X et dans y
  Renvoie X et y avec toutes les classes ayant le meme nombre d'exemple que la classe minoritaire'''
  dicoclasses = {k:v for k,v in zip(*np.unique(y, return_counts=True))}
  print("avant",dicoclasses)
  mini = min(dicoclasses.values())
  while any((v >mini for v in dicoclasses.values())):
    
    for cat in dicoclasses:
      if dicoclasses[cat] > mini:
        nb_hasard = np.random.choice(np.where(y==cat)[0],1)
        X = np.delete(X,nb_hasard, axis=0)
        y = np.delete(y,nb_hasard, axis=0)
        dicoclasses[cat]-=1

  print("apres",dicoclasses)
  return X,y
  #for i in range(len(X)):


Chargement des données

In [ ]:
dfbow = pd.read_csv(cheminbow, sep=",",index_col=0)
dfw2v = pd.read_csv(cheminw2v, sep=",",index_col=0)
dffeatures = pd.read_csv(cheminfeatures, sep=",",index_col=0)

In [ ]:
chemindata = chemincorpus+"Toutes_mentions_Defense_EN.csv"

In [ ]:
dfannot = pd.read_csv(chemindata,sep=",", index_col=0, keep_default_na=False,na_values=["NA"])
dfannot = dfannot[~(dfannot.cible_contexte.isin(["",np.NAN]))&(~dfannot.procédé_de_nommage.isin(["",np.NAN]))]
annotations = dfannot.loc[:,:"positionnement_contexte"]
annotations = annotations.fillna("NA")

In [ ]:
index_procede_rempli = annotations[annotations.procédé_de_nommage != "NA" ].index.intersection(dfbow.index).intersection(dffeatures.index)
dffeatures = dffeatures.loc[index_procede_rempli,:]
annotations = annotations.loc[index_procede_rempli,:]
dfbow = dfbow.loc[index_procede_rempli,:]
dfw2v = dfw2v.loc[index_procede_rempli,:]

In [ ]:
X_bow_mention = dfbow.iloc[:,:159]
X_bow_contexte = dfbow.iloc[:,160:]

In [ ]:
X_w2v_mention = dfw2v.iloc[:,:500]
X_w2v_contexte = dfw2v.iloc[:,501:]

# Predictions mention

In [ ]:
def validation_croisee(X,y,clf):
  y_pred = cross_val_predict(clf,X,y=y)
  report = classification_report(y,y_pred)
  print(report)

In [ ]:
def split_labels(y):
  newy = []
  for labels in y:
    labelsplit = [label.strip() for label in labels.split("/")]
    if labelsplit == ['']:
      print(labels)
    newy.append(labelsplit)
  return newy

## Procédé_nommage

### Extraction de la Norme

In [ ]:
def separe_norme(dffeatures):
  index_normes = []
  for i in dffeatures.index:
    if dffeatures.loc[i,"is_norme"]:
      index_normes.append(i)
      #print(y.iloc[i,0])
  return index_normes


In [ ]:
index_normes = separe_norme(dffeatures)

In [ ]:
df_pred = pd.DataFrame(index=annotations.index)
df_pred["procédé_de_nommage"] = "autre"
df_pred[df_pred.index.isin(index_normes)]="norme"

In [ ]:
index_not_norme = annotations[~annotations.index.isin(index_normes)].index

In [ ]:
y_norme_autre = annotations.loc[:,"procédé_de_nommage"].copy()
y_norme_autre.loc[index_not_norme] = "autre"

In [ ]:
print(classification_report(df_pred.procédé_de_nommage,y_norme_autre))

              precision    recall  f1-score   support

       autre       1.00      1.00      1.00       197
       norme       1.00      1.00      1.00       104

    accuracy                           1.00       301
   macro avg       1.00      1.00      1.00       301
weighted avg       1.00      1.00      1.00       301



In [ ]:
for elem , e in zip(df_pred.procédé_de_nommage,y_norme_autre):
  if elem != e :
    print("u")

### Autres procédés de nommage

In [ ]:
from sklearn.utils import shuffle

Préparation données

In [ ]:
index_not_norme = annotations[annotations.procédé_de_nommage!="norme"].index

In [ ]:
X_procédé = X_w2v_mention.loc[index_not_norme,:].to_numpy()
X_procédé = dffeatures.loc[index_not_norme,"contient_ordinal":"distance_norme"].to_numpy()

y_procédé = split_labels(annotations.loc[index_not_norme,"procédé_de_nommage"])

MB = MultiLabelBinarizer()
y_procédé = MB.fit_transform(y_procédé)

In [ ]:
X_procédé,y_procédé = shuffle(X_procédé,y_procédé)

#### Descripteurs Linguistiques

Symbolique

In [ ]:
y_pred_symbo = []
for ordi,super,lieu,dist_lieu,geo,dist_geo,dist_norme in  X_procédé:
  labels = [0,0,0,0]
  if ordi or super:
    labels[0] = 1
  if lieu :
    labels[3] = 1
  if geo : 
    labels[2] = 1
  if dist_geo < 0.5  and dist_lieu < 0.5 and dist_norme < 0.45:
    print(dist_lieu,dist_geo,dist_norme)
    labels[1] = 1
  y_pred_symbo.append(labels)

0.2660766541957855 0.3641175031661987 0.2859586179256439
0.4539364278316498 0.33695077896118164 0.120164155960083
0.4007566869258881 0.345424622297287 0.1265510320663452
0.4577445685863495 0.3253185749053955 0.08519981801509857


In [ ]:
report = classification_report(y_procédé,y_pred_symbo,target_names=MB.classes_)

In [ ]:
print(report)

                        precision    recall  f1-score   support

            classement       0.83      1.00      0.91         5
             métaphore       0.25      0.33      0.29         3
référence géographique       0.99      0.97      0.98       135
             type lieu       0.98      0.99      0.99       171

             micro avg       0.97      0.98      0.98       314
             macro avg       0.76      0.82      0.79       314
          weighted avg       0.98      0.98      0.98       314
           samples avg       0.98      0.98      0.98       314



Statistique

Descripteurs Linguistiques symbo

avec BoW

## Cible_mention

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import random

In [ ]:
def reequilibre_classes(X,y,cible_support):
  
  classes,counts = np.unique(np.array([label  for labels in y for label in labels]),return_counts=True)
  dico_counts = {classe: count for classe,count in zip(classes,counts)}
  dico_counts = dict(sorted(dico_counts.items(),key=lambda x:x[1] , reverse=True))
  print("avant", dico_counts)
  #print(X)

  new_X,new_y = [], []
  if not isinstance(X,list):
    X= X.tolist()
  nb_sup,nb_add = 0,0
  nb_total = len(X)
  for label in dico_counts:
    print(label,"avant", dico_counts)
    while dico_counts[label]  < cible_support   or dico_counts[label]  > cible_support:
      i = random.randint(0,len(y)-1)
      labels = y[i]
      if label not in labels : continue
      if dico_counts[label] < cible_support :
        for lab in labels:
          dico_counts[lab] +=1
        X.append(X[i])
        y.append(y[i])
        nb_add +=1
      elif dico_counts[label] > cible_support :
        if any((dico_counts[lab] < cible_support-5 for lab in labels) ): continue
        for lab in labels:
          dico_counts[lab] -=1
        del X[i]
        del y[i]
        nb_sup+=1
    
    print(label,"après", dico_counts)

  classes,counts = np.unique(np.array([label  for labels in y for label in labels]),return_counts=True)
  dico_counts = {classe: count for classe,count in zip(classes,counts)}
  print(f"exemples avant : {nb_total}", f"ajoutés : {nb_add}", f"supprimés : {nb_sup}", f"exemples après : {len(X)}", sep = " | ")
  return X,y
      



  new_dico_counts =   {classe: count for classe,count in zip(*np.unique(np.array([label  for labels in new_y for label in labels]),return_counts=True))}
  print("a la fin",new_dico_counts)

In [ ]:
X_cible_m = X_w2v_mention.loc[index_not_norme,:].to_numpy()
y_cible_m = split_labels(annotations.loc[index_not_norme,:].cible_mention)
X_cible_m, y_cible_m =reequilibre_classes(X_cible_m,y_cible_m,10)
MB = MultiLabelBinarizer()
y_cible_m = MB.fit_transform(y_cible_m)
X_cible_m,y_cible_m = shuffle(X_cible_m,y_cible_m)


avant {'économie': 147, 'localisation': 125, 'taille': 10, 'urbanisme': 7, 'architecture': 6, 'changement': 5, 'disparité': 1, 'espace de vie': 1}
économie avant {'économie': 147, 'localisation': 125, 'taille': 10, 'urbanisme': 7, 'architecture': 6, 'changement': 5, 'disparité': 1, 'espace de vie': 1}
économie après {'économie': 10, 'localisation': 51, 'taille': 4, 'urbanisme': 7, 'architecture': 4, 'changement': 4, 'disparité': 1, 'espace de vie': 1}
localisation avant {'économie': 10, 'localisation': 51, 'taille': 4, 'urbanisme': 7, 'architecture': 4, 'changement': 4, 'disparité': 1, 'espace de vie': 1}
localisation après {'économie': 6, 'localisation': 10, 'taille': 4, 'urbanisme': 4, 'architecture': 4, 'changement': 4, 'disparité': 1, 'espace de vie': 1}
taille avant {'économie': 6, 'localisation': 10, 'taille': 4, 'urbanisme': 4, 'architecture': 4, 'changement': 4, 'disparité': 1, 'espace de vie': 1}
taille après {'économie': 8, 'localisation': 14, 'taille': 10, 'urbanisme': 4, 'a

In [ ]:
MB.classes_

array(['architecture', 'changement', 'disparité', 'espace de vie',
       'localisation', 'taille', 'urbanisme', 'économie'], dtype=object)

Apprentissage

In [ ]:
#validation_croisee(X_w2v_mention.loc[index_not_norme,:].to_numpy(),annotations.loc[index_not_norme,:].procédé_de_nommage,LogisticRegression())
clf = MLPClassifier(hidden_layer_sizes=100,max_iter=400)


In [ ]:
y_pred = cross_val_predict(clf,X=X_cible_m,y=y_cible_m)


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [ ]:
report = classification_report(y_cible_m,y_pred,target_names=MB.classes_)

w2V

In [ ]:
print(report)

               precision    recall  f1-score   support

 architecture       1.00      0.95      0.97        19
   changement       1.00      1.00      1.00        10
    disparité       1.00      1.00      1.00        10
espace de vie       0.91      1.00      0.95        10
 localisation       1.00      1.00      1.00        18
       taille       1.00      0.70      0.82        10
    urbanisme       0.91      0.91      0.91        11
     économie       0.83      0.95      0.89        21

    micro avg       0.94      0.94      0.94       109
    macro avg       0.96      0.94      0.94       109
 weighted avg       0.95      0.94      0.94       109
  samples avg       0.94      0.94      0.94       109



Bag of Words

## Polarité

### Avec Textblob

In [ ]:
!pip install -U textblob-fr

     |████████████████████████████████| 561 kB 31.7 MB/s 


In [ ]:
from textblob import TextBlob

In [ ]:
y_pred_pola = []
for mention,classe in zip(mentions_pola,y_pola_m):
  doc = nlp(mention)
  for token in doc:
    blob = TextBlob(token.lemma_)
    #blob = TextBlob(mention)
    sentiments = blob.sentiment
    #print(sentiments,mention)
    pola = sentiments[0]
    if pola <0:
      
      pola_pred = "négatif"
      break
    elif pola > 0:
      
      pola_pred = "positif"
      break
    else:
      
      pola_pred ="neutre"
  if pola_pred != classe:
    print(sentiments,mention, pola_pred, "|", classe)
  y_pred_pola.append(pola_pred)

Sentiment(polarity=0.2, subjectivity=0.3) the French capital's modern business district positif | neutre
Sentiment(polarity=0.0, subjectivity=0.0) Prime office space in the centre of Paris neutre | positif
Sentiment(polarity=0.13636363636363635, subjectivity=0.45454545454545453) its new post-Brexit home positif | neutre
Sentiment(polarity=0.16, subjectivity=0.5399999999999999) the high-rise business district on the western outskirts of Paris positif | neutre
Sentiment(polarity=0.0, subjectivity=0.0) Paris's safety valve for office space neutre | positif
Sentiment(polarity=0.13636363636363635, subjectivity=0.45454545454545453) Paris's then-new business district positif | neutre


In [ ]:
print(classification_report(y_pola_m,y_pred_pola))

              precision    recall  f1-score   support

      neutre       0.99      0.98      0.98       182
     négatif       1.00      1.00      1.00         1
     positif       0.75      0.86      0.80        14

    accuracy                           0.97       197
   macro avg       0.91      0.95      0.93       197
weighted avg       0.97      0.97      0.97       197



## Positionnement mention

In [ ]:
X_posit_m = dffeatures.loc[index_not_norme,"contient_guillemet"].to_numpy()
y_posit_m = annotations.loc[index_not_norme,:].positionnement_mention.to_numpy(dtype='str')
y_posit_m = np.char.replace(np.char.strip(y_posit_m), "prise","pris")

In [ ]:
y_pred_posit_m =  []
for v in X_posit_m :
  if v:
    print(v)
    y_pred_posit_m.append("cité")
  else:
    y_pred_posit_m.append("pris en charge")



In [ ]:
report = classification_report(y_posit_m,y_pred_posit_m)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(report)

                precision    recall  f1-score   support

          cité       0.00      0.00      0.00         6
pris en charge       0.97      1.00      0.98       191

      accuracy                           0.97       197
     macro avg       0.48      0.50      0.49       197
  weighted avg       0.94      0.97      0.95       197

